In [85]:
import folium
import json
from branca.colormap import linear
import pandas as pd
from random import sample
import csv
import pyproj
import geopandas as gpd
import geojson
import numpy as np
from shapely.geometry import Point

In [43]:
with open("../out/geo_referencing_campo.json", "r", encoding='utf-8') as f:
    data = json.load(f)

In [44]:
first_numbers = [vals["years"][0] for vals in data.values() if vals["years"]]
min_val = min(first_numbers)
max_val = max(first_numbers)

colormap = linear.YlOrRd_09.scale(min_val, max_val)  # This one works in all common versions
colormap.caption = "First Number Scale"

In [47]:
# Create base map centered in Venice
venice_map = folium.Map(location=[45.4375, 12.3358], tiles="Cartodb Positron", zoom_start=14)

# Add markers
for name, info in data.items():
    if isinstance(info, dict) and info["years"] != []:
        first_number = info["years"][0]
        color = colormap(first_number)
        popup_text = f"<b>{name}</b><br><i>Year: {first_number}</i>"

        if "Campo" in name:
            folium.CircleMarker(
                location=[info["latitude"], info["longitude"]],
                radius=7,
                color=color,
                fill=True,
                fill_opacity=0.8,
                popup=popup_text,
                tooltip=name
            ).add_to(venice_map)
        else:
            folium.RegularPolygonMarker(
                location=[info["latitude"], info["longitude"]],
                radius=7,
                color=color,
                fill=True,
                fill_opacity=0.8,
                popup=popup_text,
                tooltip=name
            ).add_to(venice_map)
        
colormap.add_to(venice_map)

venice_map

In [42]:
geo_eval = pd.DataFrame(data)
geo_eval = sample(list(geo_eval.columns), 20)
geo_eval

with open('geo_reference_eval.csv', 'w') as f:
    
    write = csv.writer(f)
    
    write.writerow(["place", "true"])
    for row in geo_eval:
        write.writerow([row])

# test with streets geo data

In [133]:
with open("../out/test_geo_ref.json", "r", encoding='utf-8') as f:
    new_data = json.load(f)

In [ ]:
first_numbers = [vals["years"][0] for vals in new_data.values() if vals["years"]]
min_val = min(first_numbers)
max_val = max(first_numbers)

colormap = linear.YlOrRd_09.scale(min_val, max_val)  # This one works in all common versions
colormap.caption = "First Mention"

In [160]:
def compute_centroid(coords):
    x_sum = sum(p[0] for p in coords)
    y_sum = sum(p[1] for p in coords)
    n = len(coords)
    return (x_sum / n, y_sum / n)

In [161]:
transformer = pyproj.Transformer.from_crs("epsg:32633", "epsg:4326", always_xy=True)

In [162]:
venice_map = folium.Map(location=[45.4375, 12.3358], tiles="Cartodb Positron", zoom_start=14)

# Add markers
for name, info in new_data.items():
    if isinstance(info, dict) and info["years"] != []:
        first_number = info["years"][0]
        color = colormap(first_number)
        popup_text = f"<b>{name}</b><br><i>Year: {first_number}</i>"

        if info["location"] != {}:
            if info["location"]["type"] == "Polygon":
                coordinates = compute_centroid(info["location"]["coordinates"][0])
            elif info["location"]["type"] == "MultiPolygon":
                coordinates = compute_centroid(info["location"]["coordinates"][0][0])
            else:
                pass
            lon, lat = transformer.transform(*coordinates)  # UTM → lat/lon

            folium.CircleMarker(
                location=[lat, lon],
                radius=7,
                color=color,
                fill=True,
                fill_opacity=0.8,
                popup=popup_text,
                tooltip=name
            ).add_to(venice_map)

        
colormap.add_to(venice_map)

venice_map

# Compare within parishes

In [138]:
parishes_data = gpd.read_file("../1740_redrawn_parishes_cleaned_wikidata_standardised.geojson")

In [148]:
points_list = []


for key, info in new_data.items():
    if isinstance(info, dict) and info["years"] != []:
        if info["location"] != {}:
            if info["location"]["type"] == "Polygon":
                coordinates = compute_centroid(info["location"]["coordinates"][0])
            elif info["location"]["type"] == "MultiPolygon":
                coordinates = compute_centroid(info["location"]["coordinates"][0][0])
            else:
                pass
            lon, lat = transformer.transform(*coordinates)  # UTM → lat/lon

        points_list.append({'geometry': Point(lon, lat), 'first_year': info["years"][0]})

points_gdf = gpd.GeoDataFrame(points_list, crs=parishes_data.crs)

In [156]:
# Ensure both GeoDataFrames use the same CRS (coordinate reference system)
points_gdf = points_gdf.to_crs(parishes_data.crs)

# Spatial join: associate each point with the polygon it falls into
joined = gpd.sjoin(points_gdf, parishes_data, how='inner', predicate='within')

# Calculate average value per polygon
avg_values = joined.groupby('index_right')['first_year'].mean().round()
min_values = joined.groupby('index_right')['first_year'].min()

# Add avg_values back to polygons GeoDataFrame
parishes_data['avg_value'] = np.nan
parishes_data.loc[avg_values.index, 'avg_value'] = avg_values.values
parishes_data['min_value'] = np.nan
parishes_data.loc[min_values.index, 'min_value'] = min_values.values

In [150]:
min_val = parishes_data['avg_value'].min()
max_val = parishes_data['avg_value'].max()

colormap = linear.YlOrRd_09.scale(min_val, max_val)  # This one works in all common versions
colormap.caption = "First Number Scale"

In [ ]:
# Create base map centered roughly on your data
m = folium.Map(location=[45.4375, 12.3358], tiles="Cartodb Positron", zoom_start=14)

def style_function(feature):
    val = feature['properties']['avg_value']
    if val is None or np.isnan(val):
        return {'fillColor': '#gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.5}
    else:
        return {'fillColor': colormap(val), 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    
# Add polygons to map
folium.GeoJson(
    parishes_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['NAME', 'avg_value'], aliases=['Name', 'Average Year']),
).add_to(m)

# Add color scale legend
colormap.caption = 'Average First Mention per Parish'
colormap.add_to(m)

m


In [157]:
min_val = parishes_data['min_value'].min()
max_val = parishes_data['min_value'].max()

colormap = linear.YlOrRd_09.scale(min_val, max_val)  # This one works in all common versions
colormap.caption = "First Number Scale"

In [158]:
# Create base map centered roughly on your data
m = folium.Map(location=[45.4375, 12.3358], tiles="Cartodb Positron", zoom_start=14)

def style_function(feature):
    val = feature['properties']['min_value']
    if val is None or np.isnan(val):
        return {'fillColor': '#gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.5}
    else:
        return {'fillColor': colormap(val), 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    
# Add polygons to map
folium.GeoJson(
    parishes_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['NAME', 'min_value'], aliases=['Name', 'First Mention']),
).add_to(m)

# Add color scale legend
colormap.caption = 'First Mention of Parish'
colormap.add_to(m)

m
